## Imports

In [146]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import sys
import numpy as np
import pandas as pd
import regex as re
import requests
import lxml
from lxml.html.soupparser import fromstring
import prettify
import numbers
import htmltext
import urllib
from requests_html import HTML
from requests_html import HTMLSession
import json
from openpyxl import load_workbook
from openpyxl.styles import Font, Color, colors, fills



import warnings
warnings.filterwarnings('ignore')

## Getting all the property links in each page from get_pages_links() function

In [147]:

def get_url_links_for_props_in_ZipCode(zipCode,criteria):
    # ZipCode: String
    # criteria: String
    
    prop_links = []
    
    url = 'https://www.har.com/zipcode_{0}/realestate/{1}'.format(zipCode,criteria)
    print(url)
    req_headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,/;q=0.8',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.8',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
        }

    with requests.Session() as s:
        r = s.get(url,headers = req_headers)
        soup = BeautifulSoup(r.content,'html.parser')
        
    raw_data = soup.find_all('script',{'type':"text/javascript"})
    raw_data1 = raw_data[22]
    raw_data2 = list(raw_data1)
    end_index = raw_data2[0].rfind(';\nvar userinfo =')
    raw_data3 = raw_data2[0][20:end_index] 
    raw_data4 = json.loads(raw_data3)
    
    for i in range(len(raw_data4)):
        address_on_url = raw_data4[i]['ADDRESS_ON_URL']
        HARID2 = raw_data4[i]['HARID2']
        url = 'https://www.har.com/homedetail/{0}/{1}'.format(address_on_url,HARID2)
        prop_links.append(url)
    
    return prop_links

    
    

In [148]:
a = get_url_links_for_props_in_ZipCode('77084','for_sale')

https://www.har.com/zipcode_77084/realestate/for_sale


## Getting all the properties info from the result of get_url_links_for_props_in_ZipCode(zipCode,criteria)

In [149]:
def get_props_info_df(zipCode,criteria):
    
    req_headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,/;q=0.8',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.8',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
        }
    
    links = get_url_links_for_props_in_ZipCode(zipCode,criteria)
    
    prop_info_df = pd.DataFrame()
    
    for a in links:
        with requests.Session() as s:
            
            inter_dict = dict()
            r = s.get(a,headers = req_headers)
            soup = BeautifulSoup(r.content,'html.parser')
            soup1 = list(soup)
            soup2 = soup1[2]
            soup3 = list(soup2)
            Days_on_HAR = soup3[3].find_all('span',{'style':"font-weight:normal !important; font-size:14px;"})
            Days_on_HAR = list(Days_on_HAR[0])
            Days_on_HAR = list(Days_on_HAR[1])
            try:
                d = int(Days_on_HAR[0])
            except:
                d = int(0)
                
            Days_on_HAR = d
            
            inter_dict['Days_on_HAR'] = Days_on_HAR
            
            labels = soup.find_all('div',{'class':"dc_label"})
            values = soup.find_all('div',{'class':"dc_value"})
            
            
            for i in range(len(labels)):
                label = list(labels[i])[0]
                value = list(values[i])[0]
                inter_dict[label] = [value]
            
            df = pd.DataFrame(inter_dict)
            
            prop_info_df = prop_info_df.append(df)
            
    return prop_info_df
               

In [150]:
%%time
a = get_props_info_df('77084','for_sale')
a

https://www.har.com/zipcode_77084/realestate/for_sale
CPU times: user 32.7 s, sys: 801 ms, total: 33.5 s
Wall time: 2min 34s


,Days_on_HAR,MLS#:,Listing Price:,Listing Status:,Address:,City:,State:,Zip Code:,County:,Subdivision:,...,Garage Apt Qtr SF:,Extra Room:,Den:,Guest w/Kitchen:,Carport Description:,Exterior Porch/Balcony:,Washer/Dryer Conn:,Appliances:,Parking:,Median Appraised Value / Square ft.:
0,128,92925748 (HAR),"$ 4,499,000","[\n, [\n, [], \n, [], \n], \n, [For Sale], \n]",0 South Park View Drive,[Houston],TX,[77084],[Harris County],,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,13,26050372 (HAR),"$ 3,000,000","[\n, [\n, [], \n, [], \n], \n, [Under Contract...",0 Morton Road,[Houston],TX,[77084],[Harris County],Morton Crk Ranch,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,121,80458059 (HAR),"$ 1,307,000","[\n, [\n, [], \n, [], \n], \n, [For Sale], \n]",0 Clay Road,[Houston],TX,[77084],[Harris County],Clay,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,185,76934698 (HAR),"$ 1,200,000","[\n, [\n, [], \n, [], \n], \n, [For Sale], \n]",6603 Langham Way,[Houston],TX,[77084],[Harris County],CHURCH OF GOD HOUSE OF DAVID,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,156,22479639 (HAR),"$ 873,872","[\n, [\n, [], \n, [], \n], \n, [For Sale], \n]",0 Greenhouse Road,[Houston],TX,[77084],[Harris County],,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2,84612198 (HAR),"$ 185,000 ($179.44/sqft.)","[\n, [\n, [], \n, [], \n], \n, [For Sale], \n]",6506 Glenmorris Court,[Houston],TX,[77084],[Harris County],,...,NaN,NaN,"15 , 16, 1st",NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,6,97176781 (HAR),"$ 149,900 ($144.41/sqft.)","[\n, [\n, [], \n, [], \n], \n, [Under Contract...",5158 Beaverhollow Drive,[Houston],TX,[77084],[Harris County],,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,6,12384686 (HAR),"$ 99,900 ($115.76/sqft.)","[\n, [\n, [], \n, [], \n], \n, [Under Contract...",5235 Beaverbrook Drive,[Houston],TX,[77084],[Harris County],,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,5,94622926 (HAR),"$ 98,000 ($94.78/sqft.)","[\n, [\n, [], \n, [], \n], \n, [Under Contract...",14603 Briton Cove Drive,[Houston],TX,[77084],[Harris County],,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Cleaning up the result of get_props_info_df()

In [162]:
def clean_up_result(df_raw):
    
    df = df_raw.copy(deep = True)
    coding: utf-8 
    # columns renamed
    df.reset_index(inplace=True,drop=True)
    df.columns = df.columns.str.replace(":","")
    df.columns = df.columns.str.replace(" ","_")
    df.columns = df.columns.str.replace("#","")
    df.columns = df.columns.str.replace("0xA9","")
    df.columns = df.columns.str.replace(u'\xA9', '')
    
    for i in range(df.shape[0]):
        c = df.Listing_Status[i].find_all('div',{'class': "pull-left"})
        for tag in c:
            df.loc[i,'Listing_Status'] = tag.text
        
        try:
            df.loc[i,'Listing_Price'] = df.loc[i,'Listing_Price'].split('(')[0]
            df.loc[i,'Listing_Price'] = df.loc[i,'Listing_Price'].split('$')[1]
        except:
            df.loc[i,'Listing_Price'] = df.loc[i,'Listing_Price']
        
        df.loc[i,'City'] = df.loc[i,'City'].text
        df.loc[i,'Zip_Code'] = df.loc[i,'Zip_Code'].text
        df.loc[i,'County'] = df.loc[i,'County'].text
        df.loc[i,'Market_Area'] = df.loc[i,'Market_Area'].text
        df[['Tax_Amount','Tax_Year']] = df['Taxes_w/o_Exemp'].str.split('/',expand=True)
        
        
        
        
        try:
            if 'Acres' in df.loc[i,'Lot_Size'].text:
                df.loc[i,'Lot_Size'] = float(df.loc[i,'Lot_Size'].text.split(' ')[0].replace(',','')) * 43560
            else:
                df.loc[i,'Lot_Size'] = float(df.loc[i,'Lot_Size'].text.split(' ')[0].replace(',',''))
        except:
            pass
    
    # Other_Fees, Maintenance_Fees,Tax_Amount,Tax_Year    need attention
    # Find out what they are: Single_Family_Properties

    
    df.rename(columns={'Lot_Size':'Lot_SizeSQFT'},inplace=True)
    
    df.drop(['Key_Map','Cable','Water_Sewer','Phone','Electric','Gas','List_Type','Subdivision','Legal_Description','Lot_Description',
            'Unit_Location','Taxes_w/o_Exemp','County_/_Zip_Code','Median_Appraised_Value_/_Square_ft.','Parking','Appliances',
            'Washer/Dryer_Conn','Exterior_Porch/Balcony','Carport_Description','Guest_w/Kitchen','Den','Extra_Room',
            'Garage_Apt_Qtr_SF','Garage_Apartment','Connections'],axis=1,inplace=True)
    
    df_new = df.copy()
    
    return df_new
    

In [163]:
b = clean_up_result(a)

In [191]:
b['Connections']

0                                                 NaN
1                                                 NaN
2                                                 NaN
3                                                 NaN
4                                                 NaN
                            ...                      
209    Electric Dryer Connections, Washer Connections
210    Electric Dryer Connections, Washer Connections
211    Electric Dryer Connections, Washer Connections
212                                               NaN
213                                               NaN
Name: Connections, Length: 214, dtype: object

In [127]:
for i in b.columns:
    print(i)

Days_on_HAR
MLS
Listing_Price
Listing_Status
Address
City
State
Zip_Code
County
Property_Type
Lot_SizeSQFT
Market_Area
HOA_Mandatory
Financing_Considered
Other_Fees
Maintenance_Fee
Taxes_w/o_Exemp
Tax_Rate
Subdivision_Name
County_/_Zip_Code
Single_Family_Properties
Average_Bedrooms
Average_Baths
Median_Square_Ft.
Median_Lot_Square_Ft.
Median_Year_Built
Median_Appraised_Value
Neighborhood_Value_Range
Median_Price_/_Square_ft.
Market_Area_Name
Home_For_Sales
Average_List_Price
Average_Square_Ft.
Average_Price/Square_Ft.
Home_For_Lease
Average_Lease
Average_Lease/Square_Ft.
Lot_Dim_(F)
Lot_Dim_(B)
Lot_Dim_(L)
Lot_Dim_(R)
Road_Surface
Area_Pool
Water_Amenity
Bedrooms
Baths
Garage(s)
Stories
Style
Year_Built
Building_Sqft.
Living
Dining
Kitchen
Breakfast
Primary_Bedroom
Bedroom
Game_Room
Home_Office/Study
Media_Room
Sunroom
Room_Description
Fireplace
Floors
Kitchen_Description
Bathroom_Description
Bedroom_Description
Heating
Cooling
Microwave
Oven
Dishwasher
Ice_Maker
Compactor
Range
Dispos

In [190]:
b.to_csv('b.csv')